## Obtain Credentials (not everytime)

Typically, it is necessary to periodically renew your credentials by calling the 'ee.Authenticate()' function. Upon calling this function, a web page will open where you can sign in with your Google account. This step allows Earth Engine to authenticate your identity and associate your API requests with your specific account. After the authentication process is successfully completed, a persistent token will be stored on your local machine. 
  
By calling the 'ee.Initialize()' function in either Step 1A or 1B, the saved credentials will be automatically retrieved, allowing you to make requests through the Earth Engine Python API. This grants you the necessary privileges to access various datasets and perform spatial analysis or processings.

In [4]:
import ee
ee.Authenticate()

Enter verification code: 4/1Adeu5BXoBgBE74q0-q1yHUfXI_fVfqLKK_6wXngjog3tdlFXvn3N13OzFm4

Successfully saved authorization token.


## Step 1A: Initialize with Regular Google Account

This is the way when you want to use Google Earth Engine with your regular google account, such as your gmail account. 

In [1]:
import ee
ee.Initialize()

import geemap
import Image
import LEAFNets as LFNs

## Step 2: Define Input Parameters

To run the LEAF production tool, a number of parameters must be provided. To streamline the provision of these parameters, a Python dictionary is used as a container. Specifically, the dictionary includes TWELVE “key:value” pairs (the last three are optional), each of which is detailed as follows:
   
(1) 'sensor': a string denoting a satellite sensor and data unit. The valid values for this key currently are 'L8_SR' and 'S2_SR', representing Landsat 8/9 and Sentinel-2 surface reflectance data (for vegetation biophysical parameter estimation, the input images must be surface reflectance), respectively. 

(2) 'year' : a 4-digits integer, identifying the year of the image acquisitions (e.g., 2020).

(3) 'months' : a list of integers (1 to 12) representing the monthes of a year (e.g., [6, 7, 8] denote June, July and August). With a list of monthes, several monthly biophysical parameter products can be generated through one execution of the LEAF production tool. This list can also include only one negative integer. In this case, the biophysical parameter products corresponding to the peak season (June 15 to September 15) of a year (specified by 'year' key) will be produced.

(4) 'prod_names' : a list of biophysical parameter name strings, which can be a subset or all of the elements in ['LAI', 'fCOVER', 'Albedo', 'fAPAR', 'QC', 'date', 'partition'].

(5) 'tile_names' : a set of strings representing tile names. The regular spatial unit of the LEAF production tool is a tile, defined by the Canadian tile griding system and covering a 900km x 900km area. Providing a list of tile names allows for the creation of biophysical parameter products for multiple tiles in a single execution of the LEAF production tool. Note that to generate biophysical parameter maps for a custom region, another "key:value" pair must be added into this parameter dictionary (refer to the details on 'custom_region' key). 

(6) 'spatial_scale' : an integer that defines the spatial resolution (in meter) of the exported products.

(7) 'out_location' : a string indicating the destination for exporting the products. An acceptable value for this key is 'drive' or 'storage', corresponding to Google Drive (GD) and Google Cloud Storage (GCS), respectively.

(8) 'GCS_bucket' : a user-defined name string for a bucket on GCS. This parameter is used only when the exporting destination is GCS (when the value associated with 'out_location' is 'storage'). Note that a bucket with this specified name must exist on your GCS before exporting products. 

(9) 'out_folder' : the folder name on GD or GCS for storing the exported biophysical parameter products. If you prefer not to have the products for different tiles exported to the same directory, just leave an empty string for this key. In this case, the LEAF production tool will automatically create separate folders for the products of different tiles using directory names formed based on tile name and image acquisition year.

(10) 'custom_region': an "ee.Geometry" object that must be created with "ee.Geometry.Polygon()" function taking as input a list of Latitude and Longitude coordinates that delineate a user-defined region. This 'key: value' pair is mandatory only when a customized spatial region needs to be provided. Otherwise, DO NOT include it in this parameter dictionary, as it will overwrite the values for the 'tile_names' key. 

(11) 'start_date': a string (e.g., '2022-06-15') for specifying the beginning image acquisition date of a user-defined time period. 

(12) 'stop_date': a string (e.g., '2022-09-15') used to indicate the ending image acquisition date for a user-defined time period. Note that the string values for 'start_date' and 'stop_date' keys are also optional. They should only be supplied when a user-defined time period is required. Otherwise, OMIT these 'key:value' pairs from this parameter dictionary to prevent overwriting the values associated with 'months' key.

Of the 12 'key:value' pairs of an input parameter dictionary, nine keys ('sensor', 'year', 'spatial_scale', 'out_location', 'GCS_bucket', 'out_folder', 'custom_region', 'start_date' and 'stop_date') require a single value, while the rest three keys ('months', 'prod_names' and 'tile_names') need a list. With the combinations between the lists, various production scenarios can be carried out. For instance, to generate monthly (e.g., July and August) biophysical parameter maps for multiple tiles (e.g., 'tile41', 'tile42' and 'tile43'), two lists, [7, 8] and ['tile41', 'tile42', 'tile43'], should be specified for 'monthsfor 'tile_names' key.    

In [2]:
user_region = ee.Geometry.Polygon([[ -56.6536,  37.2523],[ -54.1060,  35.7750],[ -55.8899,  33.7511],[ -58.3861,  35.1516]])

params = {
    'sensor': 'S2_SR',           # A string indicating sensor and data unit (e.g., 'S2_SR' or 'L8_SR')    
    'year': 2023,                # An integer representing image acquisition year
    'months': [7],               # A set of integers represening months (a negative value means whole summer)
    'tile_names': ['tile55'],    # A list of tile name strings as per the CCRS' tile griding system
    'prod_names': ['LAI', 'date'],       # ['LAI','fAPAR','fCOVER','Albedo', 'QC', 'date', 'partition']
    'resolution': 20,            # The spatial resultion (in meter) of the exported products
    'location': 'drive',         # The destination of exporting the products ('drive' or 'storage')     
    'bucket': 'LEAF_production', # An unique bucket name in GCS (must have been created beforehand)
    'folder': 'S2_leaf_2023',    # The folder name intended for exporting products in either GD or GCS
    'custom_region': user_region   # A given user-defined region. Only include this 'key:value' pair as necessary
}

## Step 3: Generate and Export Biophysical Products

In [3]:
mosaic = LFNs.LEAF_production(params)

<custom_LEAF_production> func param dictionary: {'GCS_bucket': '', 'bucket': 'LEAF_production', 'custom_region': {'type': 'Polygon', 'coordinates': [[[0.9637900778919472, 50.91403368831729], [0.9702334792675978, 50.91699007337573], [1.373015219949208, 51.143035232142026], [1.392425687228554, 51.161210528531235], [1.3993416800246405, 51.17487321017443], [1.7590038910436083, 52.48232628523823], [1.7397985726778475, 52.632339348548456], [-0.7617973294684357, 60.818076114961116], [-0.7720487663815042, 60.82957167065116], [-0.84006341916535, 60.84228905763035], [-0.8825987706179472, 60.84579837326019], [-0.8989190990526906, 60.84100929771918], [-6.151035538784849, 59.10082279852729], [-8.649648277561651, 57.8312632023955], [-8.65067837292272, 57.82940821160297], [-8.181906429548674, 54.466265077464875], [-6.358374907753275, 49.891254089262475], [-6.348185796971474, 49.88672370082926], [-6.345300748028998, 49.88588533845501], [-5.207257982555416, 49.961039056211824], [-5.202317312264887, 49.

## Step 4: Check the Status of Exporting Tasks (optional)

Depending on whether you are using a regular Google account or a Google Service account to operate Google Earth Engine, there are two methods available to monitor the status of exporting tasks. If you're using a regular Google account, you can check the status by either accessing the "Tasks" tab located in the right panel of your Earth Engine Code Editor or executing the code provided in the subsequent cell. However, if you're using a Google Service account, the only way to monitor the status of exporting tasks is by running the code in the following cell.

The function "manage_tasks" requires two input parameters. The first parameter is a string and can take either 'status' or 'cancel' as its value. 'status' is used to check the status of exporting tasks, while 'cancel' is used to cancel all the exporting tasks. The second parameter is a string used to filter task names. For instance, providing '2019' will retrieve tasks with names that include '2019'. 

In [4]:
Image.manage_tasks('cancel', '2020')

tile55_2020_Jul_QC_200m has been cancelled.
tile55_2020_Jul_Albedo_200m has been cancelled.
tile55_2020_Jul_fAPAR_200m has been cancelled.
tile55_2020_Jul_fCOVER_200m has been cancelled.
tile55_2020_Jul_LAI_200m has been cancelled.
tile55_2020_Jul_QC_200m has been cancelled.
tile55_2020_Jul_Albedo_200m has been cancelled.
tile55_2020_Jul_fAPAR_200m has been cancelled.
tile55_2020_Jul_fCOVER_200m has been cancelled.
tile55_2020_Jul_LAI_200m has been cancelled.
tile55_2020_Jul_QC_200m has been cancelled.
tile55_2020_Jul_Albedo_200m has been cancelled.
tile55_2020_Jul_fAPAR_200m has been cancelled.
tile55_2020_Jul_fCOVER_200m has been cancelled.
tile55_2020_Jul_LAI_200m has been cancelled.
tile55_2020_Jul_QC_200m has been cancelled.
tile55_2020_Jul_Albedo_200m has been cancelled.
tile55_2020_Jul_fAPAR_200m has been cancelled.
tile55_2020_Jul_fCOVER_200m has been cancelled.
tile55_2020_Jul_LAI_200m has been cancelled.
tile55_2020_Jul_QC_200m has been cancelled.
tile55_2020_Jul_Albedo_200m 

## Step 1B: Initialize with Google Service Account

Initialze this way with your google service account.

In [2]:
import ee

from google.auth import compute_engine
credentials = compute_engine.Credentials(service_account_email = '751427818963-compute@developer.gserviceaccount.com', 
                                         scopes = ['https://www.googleapis.com/auth/earthengine'])

ee.Initialize(credentials, project='ccmeo-ag-000007')

import geemap
import Image
import LEAFNets as LFNs

ConnectionError: HTTPConnectionPool(host='metadata.google.internal', port=80): Max retries exceeded with url: /computeMetadata/v1/instance/service-accounts/751427818963-compute@developer.gserviceaccount.com/?recursive=true (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x000001C887EED340>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))